In [1]:
import iris
import iris.plot as iplt
import numpy as np
from iris.coords import DimCoord
from iris.coord_systems import TransverseMercator,GeogCS
from iris.cube import Cube
from cf_units import Unit
import cf_units
import os
import glob
from pyproj import Proj, transform
import sys
import warnings
import multiprocessing as mp
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

ems_hourly = ['01', '04', '06', '07', '08', '09', '10', '11', '12', '13', '15']
ems_30mins = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc013',  'bc015',  'bc016', 'bc017', 'bc018', 'bc012']
yrs_range = '1980_2001'


def mask_cube(cube, gb_mask):
    masked_cube_data = cube * gb_mask[np.newaxis, :, :]

    # APPLY THE MASK
    reshaped_mask = np.tile(gb_mask, (cube.shape[0], 1, 1))
    reshaped_mask = reshaped_mask.astype(int)
    reversed_array = ~reshaped_mask.astype(bool)

    # Mask the cube
    masked_cube = iris.util.mask_cube(cube, reversed_array)
    
    return masked_cube


### Get the mask
gb_mask_2km = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_2.2km_GB_Mask.npy")
gb_mask_12km_wgs84 = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_12km_wgs84_GB_Mask.npy")
gb_mask_12km = np.load("/nfs/a319/gy17m2a/PhD/datadir/UKCP18_12km_GB_Mask.npy")

In [2]:
##########################################################################################
#########################################################################################
# Define variables and set up environment
##########################################################################################
##########################################################################################
root_fp = "/nfs/a319/gy17m2a/"
os.chdir(root_fp)

# Create path to files containing functions
sys.path.insert(0, root_fp + 'PhD/Scripts/DataProcessing/Regridding')
from Regridding_functions import *
# Create path to files containing functions
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *

gb_gdf = create_gb_outline({'init' :'epsg:3857'})

# Load UKCP18 12km model data to use in regriddding
file_model_12km=f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/12km/01/{yrs_range}/pr_rcp85_land-rcm_uk_12km_01_day_19801201-19901130.nc'
cube_12km=iris.load_cube(file_model_12km)

file_model_2_2km ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_19910601-19910630.nc'
cube_2km =iris.load_cube(file_model_2_2km)

file_model_2_2km_bng ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng/01/1980_2001/bng_pr_rcp85_land-cpm_uk_2.2km_01_1hr_19910601-19910630.nc'
cube_2km_bng =iris.load_cube(file_model_2_2km_bng)

# remove ensemble member dimension
cube_2km = cube_2km[0,:,:,:]
cube_12km = cube_12km[0,:,:,:]

yrs_range = '1980_2001'

In [3]:
in_jja=iris.Constraint(time=lambda cell: 6 <= cell.point.month <= 8)

In [4]:
# this is the crs that we want to transform to
source_crs_2km = ccrs.RotatedGeodetic(pole_latitude=37.5,
                                        pole_longitude=177.5,
                                            central_rotated_longitude=0)

In [5]:
##################################################################
# This is a square area surrounding Leeds
leeds_at_centre_gdf = create_leeds_at_centre_outline({'init' :'epsg:3857'})
cube_12km_trimmed_to_leeds =  trim_to_bbox_of_region_obs(cube_12km, leeds_at_centre_gdf, 'projection_y_coordinate',
                                                        "projection_x_coordinate")

In [6]:
yrs_range = '1980_2001'
# ems_hourly = ['04', '06', '07', '08', '09', '10', ]

### Trim so the same size

In [ ]:
# cube_2km = trim_to_bbox_of_region_regriddedobs(cube_2km, gb_gdf)
# cube_12km = trim_to_bbox_of_region_obs(cube_12km, gb_gdf)

# iplt.contourf(cube_12km[1,:,:])
# plt.gca().coastlines(resolution='10m', color='black', linewidth=0.5);

## Transform cube 2.2km to BNG and then to 12km - masking first

In [27]:
for em in ['01', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '15']:
    print(em)
    os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em}/{yrs_range}/")
    output_fp_bng_regridded = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng_regridded_12km_maskingfirst/{em}/AreaWeighted/{yrs_range}/"
    if not os.path.isdir(output_fp_bng):
        os.makedirs(output_fp_bng)
    if not os.path.isdir(output_fp_bng_regridded):
        os.makedirs(output_fp_bng_regridded)        
    
    for filename in np.sort(glob.glob("*")):     
            if filename[47:49] in ['06', '07', '08']:
                if os.path.isfile(output_fp_bng +  f"bng_{filename}"):
                    print("already exist")                    
                else:
                    print(filename)
                    # Load the data
                    cube_2km = iris.load(filename)[0]
                    # Trim ensemble member dimension
                    cube_2km = cube_2km[0,:,:,:]
                    # Trim
                    cube_2km = trim_to_bbox_of_region_regriddedobs(cube_2km, gb_gdf)
                    # Transform to BNG
                    cube_2km_bng, lats_bng, lons_bng = convert_rotatedpol_to_bng(cube_2km.copy())
                    # Mask to GB
                    cube_2km_bng_masked = mask_cube(cube_2km_bng, gb_mask_2km)
                    # Regrid to 12km
                    cube_2km_bng_masked_regridded = cube_2km_bng_masked.regrid(cube_12km, iris.analysis.AreaWeighted()) 
                    # Save 
                    iris.save(cube_2km_bng_masked_regridded, output_fp_bng_regridded +  f"bng_rg_{filename}")     
                    


01
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19810601-19810630.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19810701-19810730.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19810801-19810830.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19820601-19820630.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19820701-19820730.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19820801-19820830.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19830601-19830630.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19830701-19830730.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19830801-19830830.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19840601-19840630.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19840701-19840730.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19840801-19840830.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19850601-19850630.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19850701-19850730.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19850801-19850830.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19860601-19860630.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19860701-19860730.nc
pr_rcp85_land-cpm_uk_2.2km_01_1hr_19860801-19860830.nc
pr_rcp8

KeyboardInterrupt: 

## Transform cube 2.2km to BNG and then to 12km

In [1]:
# for em in ['01', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '15']:
#     print(em)
#     os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em}/{yrs_range}/")
#     output_fp_bng = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng/{em}/{yrs_range}/"
#     output_fp_bng_regridded = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km_bng_regridded_12km/{em}/AreaWeighted/{yrs_range}/"
#     if not os.path.isdir(output_fp_bng):
#         os.makedirs(output_fp_bng)
#     if not os.path.isdir(output_fp_bng_regridded):
#         os.makedirs(output_fp_bng_regridded)        
#     for filename in np.sort(glob.glob("*")):     
#             if filename[47:49] in ['06', '07', '08']:
#                 if os.path.isfile(output_fp_bng +  f"bng_{filename}"):
#                     print("already exist")                    
#                 else:
#                     print(filename)
#                     cube_2km = iris.load(filename)[0]
#                     cube_2km = cube_2km[0,:,:,:]
#                     # transform to BNG
#                     cube_2km_bng= convert_rotatedpol_to_bng (cube_2km)
#                     # regrid to 12km
#                     cube_2km_regridded_12km_bng = cube_2km_bng.regrid(cube_12km, iris.analysis.AreaWeighted()) 
#                     # Save 
#                     iris.save(cube_2km_bng, output_fp_bng +  f"bng_{filename}")
#                     iris.save(cube_2km_regridded_12km_bng, output_fp_bng_regridded +  f"bng_rg_{filename}")     
                    
                    
 

## Regrid 30 mins UKCP18 data to 12km - DONE
For this need to use version of the regridding cube which is trimmed to the leeds-at-centre region, as the 30 mins data is already clipped to this extent

In [ ]:
for em in ems_30mins:
    print(em)
    os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km/{em}/2002_2020/")
    # establish paths to directories
    output_fp_bng = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{em}/{yrs_range}/"
    output_fp_bng_regridded = f"/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng_regridded_12km/{em}/AreaWeighted/{yrs_range}/"
    # create the directories
    if not os.path.isdir(output_fp_bng):
        os.makedirs(output_fp_bng)
    if not os.path.isdir(output_fp_bng_regridded):
        os.makedirs(output_fp_bng_regridded)      
    # loop through the files
    for filename in np.sort(glob.glob("*")):   
        print(filename)
        if os.path.isfile(output_fp_bng_regridded +  f"bng_rg_{filename}"):
            print("creating")
            # print(filename)
            cube_2km = iris.load(filename)[0]
            # trim 
            cube_2km = trim_to_bbox_of_region_regriddedobs(cube_2km, gb_gdf)
            # convert to bng
            cube_2km_bng, lats_bng, lons_bng = convert_rotatedpol_to_bng (cube_2km.copy()) 
            # Mask to GB
            cube_2km_bng_masked = mask_cube(cube_2km_bng, gb_mask_2km)
            # Regrid to 12km
            cube_2km_bng_masked_regridded = cube_2km_bng_masked.regrid(cube_12km, iris.analysis.AreaWeighted()) 
            # Save 
            iris.save(cube_2km_bng_masked, output_fp_bng +  f"bng_{filename}")
            iris.save(cube_2km_bng_masked_regridded, output_fp_bng_regridded +  f"bng_rg_{filename}")     
        else:
            print("already exists")    

bc005
bc005a.pr200006.nc
creating
bc005a.pr200007.nc
creating
bc005a.pr200008.nc
creating
bc005a.pr200106.nc
creating
bc005a.pr200107.nc
creating
bc005a.pr200108.nc
creating
bc005a.pr200206.nc
creating
bc005a.pr200207.nc
creating
bc005a.pr200208.nc
creating
bc005a.pr200306.nc
creating
bc005a.pr200307.nc
creating
bc005a.pr200308.nc
creating
bc005a.pr200406.nc
creating
bc005a.pr200407.nc
creating
bc005a.pr200408.nc
creating
bc005a.pr200506.nc
creating
bc005a.pr200507.nc
creating
bc005a.pr200508.nc
creating
bc005a.pr200606.nc
creating
bc005a.pr200607.nc
creating
bc005a.pr200608.nc
creating
bc005a.pr200706.nc
creating
bc005a.pr200707.nc
creating
bc005a.pr200708.nc
creating
bc005a.pr200806.nc
creating
bc005a.pr200807.nc
creating
bc005a.pr200808.nc
creating
bc005a.pr200906.nc
creating
bc005a.pr200907.nc
creating
bc005a.pr200908.nc
creating
bc005a.pr201006.nc
creating
bc005a.pr201007.nc
creating
bc005a.pr201008.nc
creating
bc005a.pr201106.nc
creating
bc005a.pr201107.nc
creating
bc005a.pr20110

## Regrid NIMROD - DONE

In [10]:
# Create version of filename specifying it is regridded
filename_to_save_to = f"rg_{filename}"

# Check if this regridded file exists, and if not create it
# Don't want to load the cube twice unnecessarily, so if we load it for 12km, then make a flag to tell us it's
# already loaded and then use this for 2.2km

# 12km regridding
try:
    print("JJA")
    cube = iris.load(filename, in_jja)[0] 
    # Fill in missing bounds
    cube.coord('projection_y_coordinate').guess_bounds()
    cube.coord('projection_x_coordinate').guess_bounds()
    # Align small rounding error in coordinates
    cube.coord('projection_x_coordinate').coord_system = cube_2km_bng.coord('projection_x_coordinate').coord_system
    cube.coord('projection_y_coordinate').coord_system = cube_2km_bng.coord('projection_y_coordinate').coord_system

    loaded_cube=True
    # Area weighted
    reg_cube_aw_2km =cube.regrid(cube_12km,iris.analysis.AreaWeighted())    
    # Save 
    iris.save(reg_cube_aw_2km, output_dir_12km + filename_to_save_to)
except:
    print("Not JJA")

# 2.2km regridding
if os.path.isfile(output_dir_2_2km + filename_to_save_to):
    print("File already exists")
if not os.path.isfile(output_dir_2_2km + filename_to_save_to):
    print('Making 2.2km file')
    if loaded_cube == False:
        try:
            cube = iris.load(filename, in_jja)[0] 
            # Area Weighted
            reg_cube_aw_12km =cube.regrid(cube_2km_bng,iris.analysis.AreaWeighted())    
            print("Regridded")
            # Save 
            iris.save(reg_cube_aw_12km, output_dir_2_2km + filename_to_save_to)    
        except:
            print("also not JJA")



loaded_cube=False


JJA
Making 2.2km file


In [ ]:
for year in range(2008,2025):
    print(year)
    # Change directory to be for correct year
    os.chdir(f"/nfs/a319/gy17m2a/PhD/datadir/NIMROD/30mins/OriginalFormat_1km/{year}")
    # Define filepaths to save files to
    output_dir_12km = f"/nfs/a319/gy17m2a/PhD/datadir/NIMROD/30mins/NIMROD_regridded_12km/AreaWeighted/{year}/"
    output_dir_2_2km = f"/nfs/a319/gy17m2a/PhD/datadir/NIMROD/30mins/NIMROD_regridded_2.2km/AreaWeighted/{year}/"
    # Create these directories if they don't exist already
    if not os.path.isdir(output_dir_12km):
        os.makedirs(output_dir_12km)
    if not os.path.isdir(output_dir_2_2km):
        os.makedirs(output_dir_2_2km)
    # Loop through all the diles in the 1km folder    
    for filename in sorted(glob.glob("*"))[1:]:
        if filename != 'metoffice-c-band-rain-radar_uk_20090728_30mins.nc':
            print(filename)
            
            # Create version of filename specifying it is regridded
            filename_to_save_to = f"rg_{filename}"

            # Check if this regridded file exists, and if not create it
            # Don't want to load the cube twice unnecessarily, so if we load it for 12km, then make a flag to tell us it's
            # already loaded and then use this for 2.2km

            # 12km regridding
            if os.path.isfile(output_dir_12km + filename_to_save_to):
                print("File already exists")
            if not os.path.isfile(output_dir_12km + filename_to_save_to):
                print('Making file 12km')
                
                try:
                    print("JJA")
                    cube = iris.load(filename, in_jja)[0] 
                    # Fill in missing bounds
                    cube.coord('projection_y_coordinate').guess_bounds()
                    cube.coord('projection_x_coordinate').guess_bounds()
                    # Align small rounding error in coordinates
                    cube.coord('projection_x_coordinate').coord_system = cube_2km_bng.coord('projection_x_coordinate').coord_system
                    cube.coord('projection_y_coordinate').coord_system = cube_2km_bng.coord('projection_y_coordinate').coord_system

                    loaded_cube=True
                    # Area weighted
                    reg_cube_aw_12km =cube.regrid(cube_12km,iris.analysis.AreaWeighted())    
                    # Save 
                    iris.save(reg_cube_aw_2km, output_dir_12km + filename_to_save_to)
                except:
                    print("Not JJA")

                # 2.2km regridding
                if os.path.isfile(output_dir_2_2km + filename_to_save_to):
                    print("File already exists")
                if not os.path.isfile(output_dir_2_2km + filename_to_save_to):
                    print('Making 2.2km file')
                    if loaded_cube == False:
                        try:
                            cube = iris.load(filename, in_jja)[0] 
                            # Area Weighted
                            reg_cube_aw_12km =cube.regrid(cube_2km_bng,iris.analysis.AreaWeighted())    
                            print("Regridded")
                            # Save 
                            iris.save(reg_cube_aw_12km, output_dir_2_2km + filename_to_save_to)    
                        except:
                            print("also not JJA")



            loaded_cube=False


2008
metoffice-c-band-rain-radar_uk_20080602_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080604_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080605_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080606_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080607_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080608_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080609_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080610_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080611_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080612_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080613_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band

Exception ignored in: <function CFReader.__del__ at 0x7f1fbd4c1680>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


also not JJA
metoffice-c-band-rain-radar_uk_20080629_30mins.nc
Making file 12km
JJA
Not JJA
Making 2.2km file


Exception ignored in: <function CFReader.__del__ at 0x7f1fbd4c1680>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


also not JJA
metoffice-c-band-rain-radar_uk_20080630_30mins.nc
Making file 12km
JJA
Not JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080701_30mins.nc
Making file 12km
JJA
Not JJA
Making 2.2km file


Exception ignored in: <function CFReader.__del__ at 0x7f1fbd4c1680>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


also not JJA
metoffice-c-band-rain-radar_uk_20080702_30mins.nc
Making file 12km
JJA
Not JJA
Making 2.2km file


Exception ignored in: <function CFReader.__del__ at 0x7f1fbd4c1680>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


also not JJA
metoffice-c-band-rain-radar_uk_20080703_30mins.nc
Making file 12km
JJA
Not JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080704_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080705_30mins.nc
Making file 12km
JJA
Not JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080706_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080707_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080708_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080709_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080710_30mins.nc
File already exists
metoffice-c-band-rain-radar_uk_20080711_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk_20080712_30mins.nc
File already exists
metoffice-c-band-rain-radar_uk_20080713_30mins.nc
Making file 12km
JJA
Making 2.2km file
metoffice-c-band-rain-radar_uk

## Regrid CEH-GEAR

### Test method with one file

In [10]:
### Read in reformatted data and set coordinate bounds
# (Might have deleted bit of code from previous versions that do the reformatting, check git history)

cube_1km = iris.load('/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_reformatted/rf_CEH-GEAR-1hr_199006.nc', in_jja)[0]
cube_1km.coord('projection_y_coordinate').guess_bounds()
cube_1km.coord('projection_x_coordinate').guess_bounds()

### Regrid to 12km using different available methods
cube_aw= cube_1km.regrid(cube_2km_bng, iris.analysis.AreaWeighted()) 
cube_nn= cube_1km.regrid(cube_2km_bng, iris.analysis.Nearest()) 
cube_l= cube_1km.regrid(cube_2km_bng, iris.analysis.Linear()) 

In [18]:
# fig, ax= plt.subplots(figsize=(15,15))
# iplt.contourf(cube_1km[15,:,:])
# plt.gca().coastlines(resolution='10m', color='black', linewidth=0.5);

### Run properly on all files

In [129]:
# contraint so we only bother for JJA data
in_jja=iris.Constraint(time=lambda cell: 6 <= cell.point.month <= 8)
# Directory where data will be saved 
dir_to_save_to = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_12km/AreaWeighted/"
# For all files in the directory
for filename in np.sort(glob.glob("/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_reformatted/*")):
    print(filename)
    filename_to_save_to = f"rg_{filename}"
    # Check if it doesn't already exist
    if not os.path.isfile(dir_to_save_to + filename_to_save_to):
        print("Creating")
        # if it's in JJA
        try:
            cube_1km = iris.load(filename, in_jja)[0] 
            print("JJA")
            # Coordinates dont have bounds
            cube_1km.coord('projection_y_coordinate').guess_bounds()
            cube_1km.coord('projection_x_coordinate').guess_bounds()
            # Regrid with area weighted
            cube_aw= cube_1km.regrid(cube_12km, iris.analysis.AreaWeighted()) 
            # File path to save to
            dir_to_save_to = f"/nfs/a319/gy17m2a/PhD/datadir/CEH-GEAR/CEH-GEAR_regridded_12km/AreaWeighted/"
            filename_to_save_to = f"rg_{filename}"
            iris.save(cube_aw, dir_to_save_to + filename_to_save_to)
        except:
            print("Not JJA")
    else:
        print("already exists")

rf_CEH-GEAR-1hr_199001.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199002.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199003.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199004.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199005.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199006.nc
Creating
JJA
rf_CEH-GEAR-1hr_199007.nc
already exists
rf_CEH-GEAR-1hr_199008.nc
Creating
JJA
rf_CEH-GEAR-1hr_199009.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199010.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199011.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199012.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199101.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199102.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199103.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199104.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199105.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199106.nc
already exists
rf_CEH-GEAR-1hr_199107.nc
Creating
JJA
rf_CEH-GEAR-1hr_199108.nc
Creating
JJA
rf_CEH-GEAR-1hr_199109.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199110.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199111.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199112.nc
Creat

Exception ignored in: <function CFReader.__del__ at 0x7f64deb683b0>
Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/iris/fileformats/cf.py", line 1263, in __del__
    self._dataset.close()
AttributeError: 'CFReader' object has no attribute '_dataset'


rf_CEH-GEAR-1hr_199908.nc
Creating
JJA
rf_CEH-GEAR-1hr_199909.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199910.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199911.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_199912.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200001.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200002.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200003.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200004.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200005.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200006.nc
Creating
JJA
rf_CEH-GEAR-1hr_200007.nc
Creating
JJA
rf_CEH-GEAR-1hr_200008.nc
Creating
JJA
rf_CEH-GEAR-1hr_200009.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200010.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200011.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200012.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200101.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200102.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200103.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200104.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200105.nc
Creating
Not JJA
rf_CEH-GEAR-1hr_200106.nc
Creating
JJA
rf_CEH-GEAR-1hr_200107.nc
Creat